### Task 0 - Data ingestion

In [ ]:
# installera nödvändiga paket till labben
%pip install duckdb pandas

In [ ]:
import duckdb
import pandas as pd 
from pathlib import Path 

# gå upp en nivå från notebooken (sql/ → projektrot) (fick hjälp av ai)
base_dir = Path("..").resolve()

# filvägar (motsvarar duckdb_path = "data/sakila.duckdb")
duckdb_path = base_dir / "data" / "sakila.duckdb"
load_sql_path = base_dir / "sql" / "load_sakila.sql"

print("duckdb:", duckdb_path)
print("load script:", load_sql_path)

In [ ]:
# radera eventuell gammal databas
duckdb_path.unlink(missing_ok=True)

# kör scriptet som laddar in Sakila-data
with duckdb.connect(duckdb_path) as conn, open(load_sql_path, encoding="utf-8") as f:
    conn.sql(f.read())
    test = conn.sql("from film limit 5;").df()

test

### Task 1 - EDA in python

In [ ]:
import duckdb
from pathlib import Path

# hitta duckdb-filen (notebook ligger i sql/, därför ..)  (fick hjälp av ai)
duckdb_path = Path("..").resolve() / "data" / "sakila.duckdb"

# öppna databasen för task 1
conn = duckdb.connect(duckdb_path)

# test
conn.sql("from film limit 3;").df()



In [ ]:
# a) filmer längre än 3 timmar

conn.sql("""
select title, length
from film
where length > 180
order by length desc;""").df()

In [ ]:
# b) filmer med love i titeln

conn.sql("""
select title
from film
where lower(title) like '%love%' 
order by title;""").df()

In [ ]:
# c) kortast, genomsnittlig, median och längsta film

conn.sql("""
select
    min(length) as shortest,
    avg(length) as average,
    median(length) as median,
    max(length) as longest
from film;""").df()

In [ ]:
# d) 10 dyraste filmerna per dag

conn.sql("""
select
    title,
    rental_rate,
    rental_duration,
    round(rental_rate / rental_duration, 2) as price_per_day
from film
where rental_duration > 0
order by price_per_day desc
limit 10;""").df()

In [ ]:
# e)